In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.vectorstores import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.runnables import RunnablePassthrough


import os
import getpass
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")
if not api_key:
    api_key = getpass.getpass("Enter your Google API Key: ")
os.environ["GOOGLE_API_KEY"] = api_key

In [ ]:
session = {}
def getSessionId(session_id: str) -> BaseChatMessageHistory:
    if session_id not in session:
        session[session_id] = ChatMessageHistory()
    return session[session_id]


In [ ]:
loader = PyPDFLoader("llm poisoning.pdf")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)
embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")
vectorstore = Chroma.from_documents(texts, embeddings)

retreiver = vectorstore.as_retriever(search_kwargs={"k": 3})

In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
system_message = """
You are a helpful assistant who only answers from the given context. If the answer is not in the context, say 'I don't know'. 
<context>
{context}
</context>
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_message),
    
])

